# Instructions:


- If names in the list have the same (example Lydia & Gary Orange) lastname take Lydia Orange as First and last name
 - if first name and last name and last name on the list are incositent (example Thomas Jacob & Caroline Kik), take Thomas Jacob as First and last name and ingnore the other names
 
 - if first name and last name and last name on the list are incositent (example Ben Janet  and Nana E. Goddy), take Ben Janet as First and last name and ingnore the other names


In [316]:
# import packages
import pandas as pd
import re

In [317]:
# Set this to True to use test data, False to use production data
TEST_DATA = True #CHANGE THIS TO False TO USE PRODUCTION DATA

sheet_name = "EPIC_Contacts_2025"

# Second cell - File Selection
if TEST_DATA:
    input_file = 'test_data.xlsx'
else:
    input_file = 'EPIC_Contacts_2025_0212.xlsx'

print(f"Using {'test' if TEST_DATA else 'production'} data")

Using test data


In [318]:

def remove_special_chars(name):
    """Preserve hyphens, apostrophes, and accented characters"""
    return re.sub(r"[\/\\[\]{}&%$@!*()~+=<>.,?]", "", name).strip()

In [319]:
def clean_name_components(name):
    """
    Clean name by removing business terms, titles, and extracting suffixes.
    """
    patterns = {
        "title_start": r"^\s*(Dr\.?|Mr\.?|Mrs\.?|Ms\.?|Prof\.?|Rev\.?|Hon\.?|Sir|Madam)\s+",
        "business": r"\b(Inc|Ltd|Co|Corp|LLC|Dba|DBA|CEO|CFO|Sr\.[ ]?Eng|Eng|C\/O)\b",
        # Improved suffix pattern with lookaheads/lookbehinds for punctuation
        "suffix": r"(?:^|\s)(Jr\.?|Sr\.?|III|IV|V|II|Ph\.?D\.?|M\.?D\.?|Esq\.?)(?=\s|$|,|&)",
    }

    # Remove business terms first
    name = re.sub(patterns["business"], "", name, flags=re.IGNORECASE)
    name = re.sub(patterns["title_start"], "", name, flags=re.IGNORECASE)

    # Find and extract the last valid suffix occurrence
    suffix_matches = list(re.finditer(patterns["suffix"], name, re.IGNORECASE))
    suffix = ""
    if suffix_matches:
        last_match = suffix_matches[-1]
        suffix = last_match.group(1)
        # Only remove the specific suffix instance we captured
        name = name[: last_match.start()] + name[last_match.end() :]
        suffix = re.sub(r"\.", "", suffix)

    # Remove middle initials with periods
    name = re.sub(r"\b([A-Z])\.", "", name)

    # Cleanup spaces and punctuation
    name = re.sub(r"\s+", " ", name).strip(" ,.&")
    return name, suffix




In [320]:
def extract_name_components(name):
    """Handle multi-person names with shared last names"""
    # Split on AND/& first
    parts = re.split(r"\s+(?:and|&)\s+", name, flags=re.IGNORECASE)
    parts = [p.strip() for p in parts if p.strip()]

    if len(parts) > 1:
        # Try to find common last name pattern (e.g. "John & Jane Smith")
        last_names = [re.findall(r"\b\w+$", p) for p in parts]
        common_last = None
        if all(len(n) == 1 for n in last_names):
            unique_last = set(n[0] for n in last_names)
            if len(unique_last) == 1:
                common_last = unique_last.pop()
                return parts[0].split()[0], common_last

        # Fallback to first complete name pair
        first_valid = next((p for p in parts if len(p.split()) >= 2), None)
        if first_valid:
            parts = first_valid.split()
            return parts[0], parts[-1]

    # Default case for single names
    words = name.split()
    if len(words) >= 2:
        return words[0], words[-1]
    return (name, "") if name else ("", "")

In [321]:
def normalize_name(name):
    """Main normalization with enhanced suffix handling"""
    if not isinstance(name, str) or not name.strip():
        return "", "", ""

    cleaned_name, suffix = clean_name_components(name)
    first_name, last_name = extract_name_components(cleaned_name)

    # Handle suffix in last name position
    suffix_keywords = {"III", "IV", "II", "Jr", "Sr"}
    if last_name in suffix_keywords:
        name_parts = cleaned_name.split()
        if len(name_parts) >= 2:
            # Preserve existing suffix if present
            new_suffix = f"{suffix} {last_name}".strip() if suffix else last_name
            suffix = new_suffix
            last_name = name_parts[-2]

    # Final cleanup while preserving valid name characters
    first_name = remove_special_chars(first_name)
    last_name = remove_special_chars(last_name)

    return first_name, last_name, suffix

In [322]:
def process_name_file(input_file, sheet_name):
    """
    Process an entire Excel file containing names and normalize them.

    Args:
        input_file (str): Path to the Excel file
        sheet_name (str): Name of the sheet containing the data

    Returns:
        pandas.DataFrame: Processed data with normalized names, or None if error
    """
    try:
        # Step 1: Load the Excel file
        print(f"Loading data from {input_file}...")
        name_data = pd.read_excel(input_file, sheet_name=sheet_name)
        total_rows = len(name_data)

        # Step 2: Apply normalization to each name in the dataset
        print("Normalizing names...")
        # Create new columns without overwriting the original DataFrame
        normalized_names = name_data["Client_Name_AMS"].apply(
            lambda x: pd.Series(
                normalize_name(str(x)), index=["FirstName", "LastName", "Suffix"]
            )
        )

        # Add the new columns to the existing DataFrame
        name_data["FirstName"] = normalized_names["FirstName"]
        name_data["LastName"] = normalized_names["LastName"]
        name_data["Suffix"] = normalized_names["Suffix"]

        # Combine Lastname and Suffix (only add space if Suffix is not empty)
        name_data["Full Lastname"] = name_data.apply(
            lambda row: f"{row['LastName']} {row['Suffix']}".strip(), axis=1
        )

        # Remove only the Suffix column
        name_data.drop(labels="Suffix", axis=1, inplace=True)

        # Step 3: Save the processed data back to Excel
        print("Saving normalized data...")
        with pd.ExcelWriter(
            input_file, engine="openpyxl", mode="a", if_sheet_exists="replace"
        ) as writer:
            name_data.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"Completed normalizing {total_rows} rows of data.")
        return name_data

    except Exception as e:
        print(f"Error processing file: {str(e)}")
        return None

In [323]:
# Main execution block
if __name__ == "__main__":
    # Set pandas display options to show all columns
    pd.set_option("display.max_columns", None)  # Show all columns
    pd.set_option("display.width", None)  # Don't wrap wide displays
    pd.set_option("display.max_rows", 15)  # Limit to 10 rows for readability

    # Process the file and get results
    result = process_name_file(input_file, sheet_name)

    # Display sample results if processing was successful
    if result is not None:
        print("\nSample of processed names:")
        print(result)

Loading data from test_data.xlsx...
Normalizing names...
Saving normalized data...
Completed normalizing 29 rows of data.

Sample of processed names:
                             Client_Name_AMS  FirstName LastName  \
0                             O'Connor James   O'Connor    James   
1                           Mary-Jane Wilson  Mary-Jane   Wilson   
2    Mr. John H. Smith Jr. & Mrs. Jane Smith       John    Smith   
3                        Lydia & Gary Orange       Gary   Orange   
4                Thomas Jacob & Caroline Kik     Thomas    Jacob   
..                                       ...        ...      ...   
24  Dr. Robert J. Wilson III and Mary Wilson     Robert   Wilson   
25               C/O John Smith & Mary Smith       John    Smith   
26                            CEO John Smith       John    Smith   
27                           CFO Jane Wilson       Jane   Wilson   
28                       Sr.Eng Robert Brown     Robert    Brown   

   Full Lastname   Expected Outpu